In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

DATA_PATH = "../../Datasets/MAMS-ATSA/Original/test/test.xml"
OUTPUT_SINGLE_ROW = "../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row.csv"
OUTPUT_MULTI_ROW = "../../Datasets/MAMS-ATSA/Downsampled/test/test_multi_row.csv"




In [2]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

#Imports the XML file and returns a DataFrame
def parse_xml_to_dataframe(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    texts = []
    aspect_terms = []

    # Iterate over each 'sentence' element
    for sentence in root.findall('./sentence'):
        text = sentence.find('text').text

        # Initialize lists to store aspect term data for the current sentence
        from_indices = []
        to_indices = []
        polarities = []
        terms = []

        # Iterate over each 'aspectTerm' element within the current 'sentence' element
        for aspect_term in sentence.findall('./aspectTerms/aspectTerm'):
            from_index = int(aspect_term.get('from'))
            to_index = int(aspect_term.get('to'))
            polarity = aspect_term.get('polarity')
            term = aspect_term.get('term')

            from_indices.append(from_index)
            to_indices.append(to_index)
            polarities.append(polarity)
            terms.append(term)

        # Append data for the current sentence to the lists
        texts.append(text)
        aspect_terms.append({'from': from_indices, 'to': to_indices, 'polarity': polarities, 'term': terms})

    # Create a DataFrame
    df = pd.DataFrame({'text': texts, 'aspect_terms': aspect_terms})
    return df
# Convert XML to DataFrame
df = parse_xml_to_dataframe(DATA_PATH)

# Print the DataFrame 
print(df.columns)
print(df.head())


Index(['text', 'aspect_terms'], dtype='object')
                                                text  \
0  The food was served promptly but the meal wasn...   
1  When I got home, there was a message on the ma...   
2  The Scene Shun Lee Palace is popular with midt...   
3  To both our surprise, this inquiry was interpr...   
4  In fact you can find their menu offerings at o...   

                                        aspect_terms  
0  {'from': [4, 13, 93], 'to': [8, 19, 103], 'pol...  
1  {'from': [64, 88, 125], 'to': [69, 96, 129], '...  
2  {'from': [4, 144], 'to': [9, 149], 'polarity':...  
3  {'from': [80, 145], 'to': [86, 153], 'polarity...  
4  {'from': [27, 89], 'to': [31, 95], 'polarity':...  


In [3]:
#Get Ramdon Sample of Dataframe
df_samples = df.sample(n=170)



#Update Index
df_samples.reset_index(drop=True, inplace=True)
print(df_samples)



                                                  text  \
0    Only thing we didn't like was the steamed choc...   
1    The food was barely decent and our server was ...   
2    We ordered whole wheast toast; the waiter didn...   
3    Our waiter brought out three main courses, but...   
4    Last, but not least, when we walked in, most o...   
..                                                 ...   
165  The food was pretty good, but due to our large...   
166  While they adhere to a certain traditionality ...   
167  The Scene Neither trendy and slick nor haute a...   
168  Had a dinner date (her suggestion) :) Had the ...   
169  After the meal, you can always walk a little s...   

                                          aspect_terms  
0    {'from': [34, 127], 'to': [70, 134], 'polarity...  
1    {'from': [4, 35], 'to': [8, 41], 'polarity': [...  
2    {'from': [35, 101], 'to': [41, 107], 'polarity...  
3    {'from': [4, 29, 117, 174], 'to': [10, 41, 121...  
4    {'from': [52,

In [4]:
#Transform DF
def transformDF(df):
    text_id = []
    term_id = []
    texts = []
    terms = []
    polarities = []
    x = 0  

    for i in range(0, len(df)):
        for j in range(0, len(df["aspect_terms"][i]["term"])):
            text_id.append(i)
            term_id.append(x)
            texts.append(df["text"][i])
            terms.append(df["aspect_terms"][i]["term"][j])
            polarities.append(df["aspect_terms"][i]["polarity"][j])
            x += 1

    df_transformed = pd.DataFrame({'text_id' : text_id, "term_id" :term_id, 'text' : texts, 'term' : terms, 'polarity' : polarities})
    return df_transformed

df_transformed = transformDF(df_samples)

print(len(df_transformed))




451


In [5]:
def print_column_counts(df, column_name):
    polarity_counts = df[column_name].value_counts()
    print(polarity_counts)
    percentage = polarity_counts / polarity_counts.sum() * 100
    print(percentage)
print(len(df_transformed))
print_column_counts(df_transformed, 'polarity')


451
polarity
neutral     206
positive    137
negative    108
Name: count, dtype: int64
polarity
neutral     45.676275
positive    30.376940
negative    23.946785
Name: count, dtype: float64


In [30]:
#df_samples.to_csv(OUTPUT_MULTI_ROW, index=False)
#df_transformed.to_csv(OUTPUT_SINGLE_ROW, index=False)    